In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext

conf = SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [2]:
lines = sc.textFile("wiki.txt")
lines.getNumPartitions()

1

In [3]:
py = sc.accumulator(0)
sp = sc.accumulator(0)

def languages(line):
    global py, sp
    if "Python" in line:
        py += 1
        if "Spark" in line:
            sp += 1
            return True
    elif "Spark" in line:
        sp += 1
        return True
    else:
        return False

In [4]:
valores = lines.filter(languages)

In [5]:
valores.count()

5

In [6]:
functionmap = valores.map(lambda x: (x,1))

In [7]:
contarvalores = functionmap.reduceByKey(lambda x,y: x+y)

In [8]:
contarvalores.count()

5

In [9]:
contarvalores.sortBy(lambda x: x[1], ascending=False).take(5)

[('Apache Spark és una plataforma de codi obert orientat', 1),
 ("L'arquitectura d'Apache Spark està basada en el concepte de RDD (Resilient Distributed Dataset), un conjunt de dades immutable distribuït al voltant d'un clúster.[4] Sobre aquesta idea fonamental, es van anar creant capes d'abstracció per a facilitar les tasques de programació i control, utilitzant per exemple el concepte de dataset (joc de dades). Així doncs, com a API es recomana la interfície orientada a datasets des de la versió Spark 2.x[5] malgrat que la orientada a RDD segueix existint.[6][7]",
  1),
 ("casos ideals per a treballar amb Spark: l'anàlisi exploratori d'un conjunt de ",
  1),
 ('Apache Spark necessitarà en qualsevol cas coordinar-se amb el clúster de ',
  1),
 ('maquinari sobre el que treballi. Per a aquesta finalitat, Spark suporta Hadoop ',
  1)]

In [13]:
def languages_map(x):
    if "Python" in x and "Spark" in x:
        return ("Count", (1,1))
    elif "Python" in x:
        return ("Count", (1, 0))
    elif "Spark" in x:
        return ("Count", (0,1))
    else:
        return ("Count", (0,0))
    
mapfun = lines.map(languages_map)

In [14]:
mapfun.count()

40

In [15]:
mapfun.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).collect()

[('Count', (0, 5))]